<img src="https://csshint.com/wp-content/uploads/2019/05/Animated-Logo-examples-2.gif">

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("../input/stackindex/MLTollsStackOverflow.csv")

In [ ]:
data.head()

In [ ]:
data.tail()

Consist of various columns containing names of libraries and the rows having the count of questions in StackOverflow for each month from 2009 to 2019

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from pandas.plotting import autocorrelation_plot
from pandas.plotting import lag_plot

In [ ]:
def interactive_plot(df, title):
    fig = px.line(title = title)
    for i in df.columns[1:]:
        fig.add_scatter(x = df['month'], y = df[i], name = i)
    fig.show()

In [ ]:
interactive_plot(data,'Stackoverflow Questions Count over years')

In [ ]:
def normalize(df):
    x = df.copy()
    for i in x.columns[1:]:
        x[i] = x[i]/x[i][0]
    return x

In [ ]:
interactive_plot(normalize(data),'Stackoverflow Questions Count over years')

In [ ]:
import seaborn as sns
correlation = data.corr()
plt.figure(figsize = (12 , 12))
sns.heatmap(correlation)

In [ ]:

f = plt.figure(figsize=(40,20))
ax = plt.gca()
data.hist(ax=ax,figsize=(40,20))
plt.show()

In [ ]:
tria=data.copy()

In [ ]:
tria.drop('Tableau',axis=1,inplace=True)

In [ ]:
for i in tria.index:
    tria.loc[i,'year']=tria['month'].str.split('-')[i][0]
    tria.loc[i,'months']=tria['month'].str.split('-')[i][1]

In [ ]:
tria['year']=tria['year'].astype(int)


In [ ]:
tria.drop('months',axis=1,inplace=True)
tria.drop('month',axis=1,inplace=True)

In [ ]:
lag_plot(tria)
plt.show()

Current observation shows strong positive correlation with previous observations.

In [ ]:
 tria.plot.hexbin(x='year', y='python', gridsize=20)

In [ ]:
tria.boxplot(figsize=(15,6),by='year',column='python')

In [ ]:
data_topics =  data.drop(columns=["month"])
print(data_topics.head())

In [ ]:
total= data_topics.sum(axis = 0)
top_tech = total.sort_values(ascending = False)[0:11]


In [ ]:
top_tech

In [ ]:
top_tech_dataframe=pd.DataFrame(top_tech,columns=['Sum'])
top_tech_dataframe.style.background_gradient(cmap='PuBu', low=0, high=0, axis=0, subset=None)

In [ ]:
fig = px.pie(top_tech_dataframe,values=top_tech_dataframe['Sum'],names=top_tech_dataframe.index,title='Top Tech')
fig.show()

In [ ]:
autocorrelation_plot(top_tech_dataframe)

In [ ]:
fig = px.sunburst(data, path=['month'],values='python-3.x',color ='python-3.x')
fig.show()

In [ ]:
data.index

In [ ]:
for i in data.index:
    data.loc[i,'year']=data['month'].str.split('-')[i][0]
    data.loc[i,'months']=data['month'].str.split('-')[i][1]
    

In [ ]:
a = ['r', 'python']
fig = px.scatter_3d(data, x='year', y='months', z='numpy', color='year')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=data['year'],
    y=data['python'],
    name='Python',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=data['year'],
    y=data['r'],
    name='r',
    marker_color='lightsalmon'
))
fig.add_trace(go.Bar(
    x=data['year'],
    y=data['pandas'],
    name='pandas',
    marker_color='green'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45,title_text="Count of Questions from the topics over years",xaxis_title="Years",
    yaxis_title="Count")
fig.show()

In [ ]:
fig = go.Figure(go.Bar(x=data['year'], y=data['python'], name='python'))
fig.add_trace(go.Bar(x=data['year'], y=data['machine-learning'], name='machine-learning'))
fig.add_trace(go.Bar(x=data['year'], y=data['r'], name='r'))

fig.update_layout(barmode='stack',xaxis_tickangle=-45,title_text="Count of Questions from the topics over years",xaxis_title="Years",
    yaxis_title="Count")
fig.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationary(timeseries):
    #determine rolling statistics
    rolmean=timeseries.rolling(12).mean()
    rolstd=timeseries.rolling(12).std()
    orig=plt.plot(timeseries,color='blue',label='Original')
    mean=plt.plot(rolmean,color='red',label='mean')
    std=plt.plot(rolstd,color='black',label='rolling std')
    plt.legend(loc='best')
    plt.title('Rolling mean and standard deviation')
    plt.show(block=False)
    print("Result of Dickey Fuler Test:")
    dftest=adfuller(timeseries,autolag='AIC')
    dfoutput=pd.Series(dftest[0:4],index=['Test Statistics','n_lags','p-value','nobs'])
    
    for key,value in dftest[4].items():
        
        dfoutput['Critical Values(%s)'%key]=value
    print (dfoutput)

In [ ]:
ts=data['python']

In [ ]:
plt.plot(ts)

It is clearly evident that there is an overall increasing trend in the data along with some seasonal variations. However, it might not always be possible to make such visual inferences (we’ll see such cases later). 

Null Hypothesis-Timeseries is non stationary
Alternate Hypothesis-Timeseries is stationary

In [ ]:
test_stationary(ts)

test statistic>critical values so we accept the null hypothesis i.e Timeseries is non-stationary

**If You liked it,Please Upvote!!**

<img src="https://i.pinimg.com/originals/7c/3b/63/7c3b63598dc8b65b93a9532d4228947b.gif">